In [ ]:
from google.colab import drive
import os

drive.mount("/content/drive")

%cd /content/drive/MyDrive/EPITA/SCIA/ing3/DNN/

Mounted at /content/drive
/content/drive/MyDrive/EPITA/SCIA/ing3/DNN


In [ ]:
!pip install transformers

In [ ]:
from model import BertForTokenAndSequenceJointClassification
import torch
from transformers import BertTokenizerFast

device = device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
model = BertForTokenAndSequenceJointClassification.from_pretrained(
    "QCRI/PropagandaTechniquesAnalysis-en-BERT",
    revision="v0.1.0",
).to(device)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [ ]:
def classify_text(text: str):
    inputs = tokenizer.encode_plus(text, return_tensors="pt").to(device)
    outputs = model(**inputs)

    token_class_index = torch.argmax(outputs.token_logits, dim=-1)
    tags = [model.token_tags[i] for i in token_class_index[0].tolist()[1:-1]]
    print(tags)

    sequence_class_index = torch.argmax(outputs.sequence_logits, dim=-1)
    sequence_class = model.sequence_tags[sequence_class_index[0]]
    print(sequence_class)

In [ ]:
classify_text('You are weak go to the gym.')

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Prop


In [ ]:
PROMPT = "Tu es une bernard de penser"
inputs = tokenizer.encode_plus(PROMPT, return_tensors="pt")
inputs

{'input_ids': tensor([[  101, 17037, 13936, 25731,  1129, 11782,  2956,  1260,  8228,  6906,
           102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
outputs = model(**inputs)
outputs

TokenAndSequenceJointClassifierOutput(loss=None, token_logits=tensor([[[-4.7391e-01,  1.2727e+00, -7.9648e-02, -2.0432e-01,  2.7146e-01,
          -6.9243e-01, -5.3114e-01,  1.0409e+00,  1.9357e-01, -5.9476e-01,
          -7.6223e-02,  2.5848e-01, -6.0015e-01, -7.9714e-02, -2.4851e-01,
          -5.8257e-02,  3.7232e-03, -7.6446e-02,  1.5029e-01,  5.1085e-01],
         [-4.8584e+00,  8.3093e+00, -6.5503e-01, -6.8260e-01, -5.6037e-01,
          -5.7799e-01, -4.0027e-01, -9.0659e-01,  4.8813e-01, -5.6304e-01,
          -1.8708e+00, -2.1549e+00, -1.0170e+00, -1.9315e+00, -6.1999e-01,
          -7.3837e-01, -1.3753e+00, -1.4498e+00, -1.9136e+00, -1.3835e+00],
         [-5.0118e+00,  8.8233e+00, -5.6928e-01, -8.6940e-01, -8.8419e-01,
          -6.8389e-01, -3.1527e-01, -7.7740e-01,  3.1317e-01, -3.4399e-01,
          -1.8475e+00, -2.2494e+00, -1.0876e+00, -2.0080e+00, -6.7390e-01,
          -1.0144e+00, -1.4175e+00, -1.5429e+00, -1.9385e+00, -1.2985e+00],
         [-5.0282e+00,  8.8256e+00,

In [ ]:
sequence_class_index = torch.argmax(outputs.sequence_logits, dim=-1)
sequence_class_index

tensor([0])

In [ ]:
sequence_class = model.sequence_tags[sequence_class_index[0]]
sequence_class

'Non-prop'

In [ ]:
token_class_index = torch.argmax(outputs.token_logits, dim=-1)
token_class_index

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(inputs.input_ids[0][1:-1])
tokens

['Tu', 'es', 'une', 'be', '##rna', '##rd', 'de', 'pen', '##ser']

In [ ]:
tags = [model.token_tags[i] for i in token_class_index[0].tolist()[1:-1]]
tags

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']